In [16]:
!

In [1]:
from gurobipy import Model, GRB
m = Model("graph_coloring")
def read_input(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()        
        num_vertices, num_interference_edges, num_affinity_edges = map(int, lines[0].split())       
        E = [tuple(map(int, line.split())) for line in lines[1:1+num_interference_edges]]        
        A = [tuple(map(int, line.split())) for line in lines[1+num_interference_edges:]]     
        V = list(range(1, num_vertices + 1))
        return V, E, A
V, E, A = read_input("sample_1.txt")
K = len(V)  
x = {}  
for v in V:
    for k in range(K):
        x[v, k] = m.addVar(vtype=GRB.BINARY, name=f"x_{v}_{k}")
y = {}  
for k in range(K):
    y[k] = m.addVar(vtype=GRB.BINARY, name=f"y_{k}")    
m.setObjective(sum(y[k] for k in range(K)), GRB.MINIMIZE)
for v in V:
    m.addConstr(sum(x[v, k] for k in range(K)) == 1)
for u, v in E:
    for k in range(K):
        m.addConstr(x[u, k] + x[v, k] <= 1)
for v in V:
    for k in range(K):
        m.addConstr(x[v, k] <= y[k])
m.optimize()

chromatic_number = sum(1 for k in range(K) if y[k].X > 0.5)
print(f"Chromatic Number: {chromatic_number}")


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-16
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1800 rows, 342 columns and 3888 nonzeros
Model fingerprint: 0x007dcb1d
Variable types: 0 continuous, 342 integer (342 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 13.0000000
Presolve removed 1588 rows and 72 columns
Presolve time: 0.03s
Presolved: 212 rows, 270 columns, 1638 nonzeros
Variable types: 0 continuous, 270 integer (270 binary)
Found heuristic solution: objective 12.0000000

Root relaxation: objective 7.000000e+00, 166 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node

In [6]:
m = Model("max_affinity_edges")
x_vars = {}
for v in V:
    for k in range(chromatic_number):
        x_vars[v, k] = m.addVar(vtype=GRB.BINARY, name=f"x_{v}_{k}")
z = {}
for a, b in A:
    z[a, b] = m.addVar(vtype=GRB.BINARY, name=f"z_{a}_{b}")
for v in V:
    m.addConstr(sum(x_vars[v, k] for k in range(chromatic_number)) == 1)
for u, v in E:
    for k in range(chromatic_number):
        m.addConstr(x_vars[u, k] + x_vars[v, k] <= 1)
for a, b in A:
    m.addConstr(sum(x_vars[a, k] * x_vars[b, k] for k in range(chromatic_number)) >= z[a, b])
m.setObjective(sum(z[a, b] for a, b in A), GRB.MAXIMIZE)
m.optimize()
if m.status == GRB.OPTIMAL:    
    color_assignment = {}
    for v in V:
        for k in range(chromatic_number):
            if x_vars[v, k].X > 0.5:
                color_assignment[v] = k
                break   
    for v in V:
        color = color_assignment.get(v, "No color assigned")
        print(f"Vertex {v} is assigned color {color}")    
    satisfied_count = 0
    print("\nSatisfied Affinity Edges:")
    for a, b in A:
        if z[a, b].X > 0.5:
            satisfied_count += 1
            print(f"Edge ({a}, {b}) is satisfied with color {color_assignment.get(a)}")
        else:
            print(f"Edge ({a}, {b}) is not satisfied")  
    print(f"\nTotal Number of Satisfied Affinity Edges: {satisfied_count}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 585 rows, 138 columns and 1260 nonzeros
Model fingerprint: 0x4d9b06e4
Model has 12 quadratic constraints
Variable types: 0 continuous, 138 integer (138 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 434 rows and 0 columns
Presolve time: 0.01s
Presolved: 331 rows, 222 columns, 1300 nonzeros
Variable types: 0 continuous, 222 integer (222 binary)
Found heuristic solution: objective 9.0000000

Root relaxation: objective 1.200000e+01, 252 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     W

In [7]:
m = Model("graph_coloring")
def read_input(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()        
        num_vertices, num_interference_edges, num_affinity_edges = map(int, lines[0].split())       
        E = [tuple(map(int, line.split())) for line in lines[1:1+num_interference_edges]]       
        A = [tuple(map(int, line.split())) for line in lines[1+num_interference_edges:]]        
        V = list(range(1, num_vertices + 1))
        return V, E, A
V, E, A = read_input("sample_2.txt")
K = len(V) 
x = {} 
for v in V:
    for k in range(K):
        x[v, k] = m.addVar(vtype=GRB.BINARY, name=f"x_{v}_{k}")
y = {}  
for k in range(K):
    y[k] = m.addVar(vtype=GRB.BINARY, name=f"y_{k}")    
m.setObjective(sum(y[k] for k in range(K)), GRB.MINIMIZE)
for v in V:
    m.addConstr(sum(x[v, k] for k in range(K)) == 1)
for u, v in E:
    for k in range(K):
        m.addConstr(x[u, k] + x[v, k] <= 1)
for v in V:
    for k in range(K):
        m.addConstr(x[v, k] <= y[k])
m.optimize()
chromatic_number = sum(1 for k in range(K) if y[k].X > 0.5)
print(f"Chromatic Number: {chromatic_number}")


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 310 rows, 110 columns and 700 nonzeros
Model fingerprint: 0x47712155
Variable types: 0 continuous, 110 integer (110 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 7.0000000
Presolve removed 263 rows and 30 columns
Presolve time: 0.00s
Presolved: 47 rows, 80 columns, 260 nonzeros
Variable types: 0 continuous, 80 integer (80 binary)
Found heuristic solution: objective 6.0000000

Root relaxation: objective 4.000000e+00, 50 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   G

In [8]:
m = Model("max_affinity_edges")
x_vars = {}
for v in V:
    for k in range(chromatic_number):
        x_vars[v, k] = m.addVar(vtype=GRB.BINARY, name=f"x_{v}_{k}")
z = {}
for a, b in A:
    z[a, b] = m.addVar(vtype=GRB.BINARY, name=f"z_{a}_{b}")
for v in V:
    m.addConstr(sum(x_vars[v, k] for k in range(chromatic_number)) == 1)
for u, v in E:
    for k in range(chromatic_number):
        m.addConstr(x_vars[u, k] + x_vars[v, k] <= 1)
for a, b in A:
    m.addConstr(sum(x_vars[a, k] * x_vars[b, k] for k in range(chromatic_number)) >= z[a, b])
m.setObjective(sum(z[a, b] for a, b in A), GRB.MAXIMIZE)
m.optimize()
if m.status == GRB.OPTIMAL:
   
    color_assignment = {}
    for v in V:
        for k in range(chromatic_number):
            if x_vars[v, k].X > 0.5:
                color_assignment[v] = k
                break 
    for v in V:
        color = color_assignment.get(v, "No color assigned")
        print(f"Vertex {v} is assigned color {color}")
    satisfied_count = 0
    print("\nSatisfied Affinity Edges:")
    for a, b in A:
        if z[a, b].X > 0.5:
            satisfied_count += 1
            print(f"Edge ({a}, {b}) is satisfied with color {color_assignment.get(a)}")
        else:
            print(f"Edge ({a}, {b}) is not satisfied")   
    print(f"\nTotal Number of Satisfied Affinity Edges: {satisfied_count}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 90 rows, 47 columns and 200 nonzeros
Model fingerprint: 0xa875eb8c
Model has 7 quadratic constraints
Variable types: 0 continuous, 47 integer (47 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 52 rows and 0 columns
Presolve time: 0.00s
Presolved: 101 rows, 75 columns, 283 nonzeros
Variable types: 0 continuous, 75 integer (75 binary)
Found heuristic solution: objective 6.0000000

Root relaxation: objective 7.000000e+00, 69 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Un

In [9]:
m = Model("graph_coloring")
def read_input(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()       
        num_vertices, num_interference_edges, num_affinity_edges = map(int, lines[0].split())      
        E = [tuple(map(int, line.split())) for line in lines[1:1+num_interference_edges]]     
        A = [tuple(map(int, line.split())) for line in lines[1+num_interference_edges:]]       
        V = list(range(1, num_vertices + 1))
        return V, E, A
V, E, A = read_input("sample_3.txt")
K = len(V) 
x = {}  
for v in V:
    for k in range(K):
        x[v, k] = m.addVar(vtype=GRB.BINARY, name=f"x_{v}_{k}")
y = {}  
for k in range(K):
    y[k] = m.addVar(vtype=GRB.BINARY, name=f"y_{k}")  
m.setObjective(sum(y[k] for k in range(K)), GRB.MINIMIZE)
for v in V:
    m.addConstr(sum(x[v, k] for k in range(K)) == 1)
for u, v in E:
    for k in range(K):
        m.addConstr(x[u, k] + x[v, k] <= 1)
for v in V:
    for k in range(K):
        m.addConstr(x[v, k] <= y[k])
m.optimize()
chromatic_number = sum(1 for k in range(K) if y[k].X > 0.5)
print(f"Chromatic Number: {chromatic_number}")


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 75 rows, 30 columns and 165 nonzeros
Model fingerprint: 0x53249f41
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5.0000000
Presolve removed 66 rows and 10 columns
Presolve time: 0.00s
Presolved: 9 rows, 20 columns, 40 nonzeros
Found heuristic solution: objective 4.0000000
Variable types: 0 continuous, 20 integer (20 binary)

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 4 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, be

In [10]:
m = Model("max_affinity_edges")
x_vars = {}
for v in V:
    for k in range(chromatic_number):
        x_vars[v, k] = m.addVar(vtype=GRB.BINARY, name=f"x_{v}_{k}")
z = {}
for a, b in A:
    z[a, b] = m.addVar(vtype=GRB.BINARY, name=f"z_{a}_{b}")
for v in V:
    m.addConstr(sum(x_vars[v, k] for k in range(chromatic_number)) == 1)
for u, v in E:
    for k in range(chromatic_number):
        m.addConstr(x_vars[u, k] + x_vars[v, k] <= 1)
for a, b in A:
    m.addConstr(sum(x_vars[a, k] * x_vars[b, k] for k in range(chromatic_number)) >= z[a, b])
m.setObjective(sum(z[a, b] for a, b in A), GRB.MAXIMIZE)
m.optimize()
if m.status == GRB.OPTIMAL:
    color_assignment = {}
    for v in V:
        for k in range(chromatic_number):
            if x_vars[v, k].X > 0.5:
                color_assignment[v] = k
                break

    for v in V:
        color = color_assignment.get(v, "No color assigned")
        print(f"Vertex {v} is assigned color {color}")
    satisfied_count = 0
    print("\nSatisfied Affinity Edges:")
    for a, b in A:
        if z[a, b].X > 0.5:
            satisfied_count += 1
            print(f"Edge ({a}, {b}) is satisfied with color {color_assignment.get(a)}")
        else:
            print(f"Edge ({a}, {b}) is not satisfied")
    print(f"\nTotal Number of Satisfied Affinity Edges: {satisfied_count}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 41 rows, 21 columns and 92 nonzeros
Model fingerprint: 0x648e9af3
Model has 1 quadratic constraint
Variable types: 0 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 28 rows and 0 columns
Presolve time: 0.00s
Presolved: 22 rows, 25 columns, 73 nonzeros
Variable types: 0 continuous, 25 integer (25 binary)
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (toler

In [11]:
m = Model("graph_coloring")
def read_input(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        num_vertices, num_interference_edges, num_affinity_edges = map(int, lines[0].split())
        E = [tuple(map(int, line.split())) for line in lines[1:1+num_interference_edges]]
        A = [tuple(map(int, line.split())) for line in lines[1+num_interference_edges:]]
        V = list(range(1, num_vertices + 1))
        return V, E, A
V, E, A = read_input("sample_4.txt")
K = len(V)
x = {}
for v in V:
    for k in range(K):
        x[v, k] = m.addVar(vtype=GRB.BINARY, name=f"x_{v}_{k}")
y = {}  
for k in range(K):
    y[k] = m.addVar(vtype=GRB.BINARY, name=f"y_{k}")
m.setObjective(sum(y[k] for k in range(K)), GRB.MINIMIZE)
for v in V:
    m.addConstr(sum(x[v, k] for k in range(K)) == 1)
for u, v in E:
    for k in range(K):
        m.addConstr(x[u, k] + x[v, k] <= 1)
for v in V:
    for k in range(K):
        m.addConstr(x[v, k] <= y[k])
m.optimize()
chromatic_number = sum(1 for k in range(K) if y[k].X > 0.5)
print(f"Chromatic Number: {chromatic_number}")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1900 rows, 420 columns and 4160 nonzeros
Model fingerprint: 0xfefd1b43
Variable types: 0 continuous, 420 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 15.0000000
Presolve removed 1669 rows and 180 columns
Presolve time: 0.03s
Presolved: 231 rows, 240 columns, 1440 nonzeros
Variable types: 0 continuous, 240 integer (240 binary)
Found heuristic solution: objective 8.0000000

Root relaxation: objective 5.000000e+00, 255 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent   

In [12]:
m = Model("max_affinity_edges")
x_vars = {}
for v in V:
    for k in range(chromatic_number):
        x_vars[v, k] = m.addVar(vtype=GRB.BINARY, name=f"x_{v}_{k}")
z = {}
for a, b in A:
    z[a, b] = m.addVar(vtype=GRB.BINARY, name=f"z_{a}_{b}")
for v in V:
    m.addConstr(sum(x_vars[v, k] for k in range(chromatic_number)) == 1)
for u, v in E:
    for k in range(chromatic_number):
        m.addConstr(x_vars[u, k] + x_vars[v, k] <= 1)
for a, b in A:
    m.addConstr(sum(x_vars[a, k] * x_vars[b, k] for k in range(chromatic_number)) >= z[a, b])
m.setObjective(sum(z[a, b] for a, b in A), GRB.MAXIMIZE)
m.optimize()
if m.status == GRB.OPTIMAL:
    color_assignment = {}
    for v in V:
        for k in range(chromatic_number):
            if x_vars[v, k].X > 0.5:
                color_assignment[v] = k
                break
    for v in V:
        color = color_assignment.get(v, "No color assigned")
        print(f"Vertex {v} is assigned color {color}")
    satisfied_count = 0
    print("\nSatisfied Affinity Edges:")
    for a, b in A:
        if z[a, b].X > 0.5:
            satisfied_count += 1
            print(f"Edge ({a}, {b}) is satisfied with color {color_assignment.get(a)}")
        else:
            print(f"Edge ({a}, {b}) is not satisfied")
    print(f"\nTotal Number of Satisfied Affinity Edges: {satisfied_count}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 390 rows, 119 columns and 840 nonzeros
Model fingerprint: 0x323020be
Model has 19 quadratic constraints
Variable types: 0 continuous, 119 integer (119 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 225 rows and 0 columns
Presolve time: 0.01s
Presolved: 374 rows, 214 columns, 1234 nonzeros
Variable types: 0 continuous, 214 integer (214 binary)
Found heuristic solution: objective 12.0000000
Found heuristic solution: objective 14.0000000

Root relaxation: objective 1.900000e+01, 265 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Curr

In [13]:
m = Model("graph_coloring")
def read_input(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        num_vertices, num_interference_edges, num_affinity_edges = map(int, lines[0].split())
        E = [tuple(map(int, line.split())) for line in lines[1:1+num_interference_edges]]
        A = [tuple(map(int, line.split())) for line in lines[1+num_interference_edges:]]
        V = list(range(1, num_vertices + 1))
        return V, E, A
V, E, A = read_input("sample_5.txt")
K = len(V)  
x = {}  
for v in V:
    for k in range(K):
        x[v, k] = m.addVar(vtype=GRB.BINARY, name=f"x_{v}_{k}")
y = {}  
for k in range(K):
    y[k] = m.addVar(vtype=GRB.BINARY, name=f"y_{k}")
m.setObjective(sum(y[k] for k in range(K)), GRB.MINIMIZE)
for v in V:
    m.addConstr(sum(x[v, k] for k in range(K)) == 1)
for u, v in E:
    for k in range(K):
        m.addConstr(x[u, k] + x[v, k] <= 1)
for v in V:
    for k in range(K):
        m.addConstr(x[v, k] <= y[k])
m.optimize()
chromatic_number = sum(1 for k in range(K) if y[k].X > 0.5)
print(f"Chromatic Number: {chromatic_number}")


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 980 rows, 210 columns and 2128 nonzeros
Model fingerprint: 0x18b24b1b
Variable types: 0 continuous, 210 integer (210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 11.0000000
Presolve removed 960 rows and 126 columns
Presolve time: 0.01s
Presolved: 20 rows, 84 columns, 168 nonzeros
Found heuristic solution: objective 6.0000000
Variable types: 0 continuous, 84 integer (84 binary)

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 6 11 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.000000000

In [14]:
m = Model("max_affinity_edges")
x_vars = {}
for v in V:
    for k in range(chromatic_number):
        x_vars[v, k] = m.addVar(vtype=GRB.BINARY, name=f"x_{v}_{k}")
z = {}
for a, b in A:
    z[a, b] = m.addVar(vtype=GRB.BINARY, name=f"z_{a}_{b}")
for v in V:
    m.addConstr(sum(x_vars[v, k] for k in range(chromatic_number)) == 1)
for u, v in E:
    for k in range(chromatic_number):
        m.addConstr(x_vars[u, k] + x_vars[v, k] <= 1)
for a, b in A:
    m.addConstr(sum(x_vars[a, k] * x_vars[b, k] for k in range(chromatic_number)) >= z[a, b])
m.setObjective(sum(z[a, b] for a, b in A), GRB.MAXIMIZE)
m.optimize()
if m.status == GRB.OPTIMAL:
    color_assignment = {}
    for v in V:
        for k in range(chromatic_number):
            if x_vars[v, k].X > 0.5:
                color_assignment[v] = k
                break
    for v in V:
        color = color_assignment.get(v, "No color assigned")
        print(f"Vertex {v} is assigned color {color}")
    satisfied_count = 0
    print("\nSatisfied Affinity Edges:")
    for a, b in A:
        if z[a, b].X > 0.5:
            satisfied_count += 1
            print(f"Edge ({a}, {b}) is satisfied with color {color_assignment.get(a)}")
        else:
            print(f"Edge ({a}, {b}) is not satisfied")
    print(f"\nTotal Number of Satisfied Affinity Edges: {satisfied_count}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 344 rows, 92 columns and 744 nonzeros
Model fingerprint: 0x80d23d1d
Model has 8 quadratic constraints
Variable types: 0 continuous, 92 integer (92 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 276 rows and 0 columns
Presolve time: 0.00s
Presolved: 172 rows, 140 columns, 656 nonzeros
Variable types: 0 continuous, 140 integer (140 binary)
Found heuristic solution: objective 8.0000000

Explored 1 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 8 

Optimal solution fou